In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
csv_path = "final.csv"
image_path = "./images"
batch_size = 256
epochs = 200
learning_rate = 1e-3
seed = 42

In [3]:
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [4]:
class ImageDataset(Dataset):
    def __init__(self, csv_path, image_path, image_transform=transforms.Compose([transforms.ToTensor()])):
        super(ImageDataset).__init__()
        csv = pd.read_csv(csv_path)
        csv_np = csv.to_numpy()
        self.images = []
        for Id in csv['Id']:
            Id = str(Id)
            image = Image.open(image_path+'/'+Id+'.jpg')
            self.images.append(image_transform(image))
            image.close()
        
        self.target = torch.from_numpy(np.array(csv_np[:,[12]], dtype="float")).float()
    
    
    def __getitem__(self,idx):
        return self.images[idx], self.target[idx]
    
    
    def __len__(self):
        return len(self.images)

In [5]:
data_length = len(pd.read_csv(csv_path))
train_length = int(data_length * 0.6)
test_length = int(data_length * 0.2)
val_length = data_length - train_length - test_length

image_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5),
                                                         (0.5, 0.5, 0.5))
                                     ])

train_dataset = ImageDataset(csv_path, image_path, image_transform)
train_dataset, test_dataset = random_split(train_dataset, [train_length, test_length+val_length])
test_dataset, val_dataset = random_split(test_dataset, [test_length, val_length])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = True)

In [6]:
class ImageNet(nn.Module):
    def __init__(self):
        super(ImageNet, self).__init__()
        self.resnet18 = torchvision.models.resnet18(pretrained=False)
        self.resnet18.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet18.fc = nn.Sequential(
        nn.Linear(512, 1000),
        nn.ReLU(),
        nn.Linear(1000, 1)
        )
        self.resnet18.to(device)
    
    def forward(self, x):
        return self.resnet18(x)

In [7]:
model = ImageNet().to(device)

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
for epoch in range(epochs):
    model.train()
    criterion.train()
    
    avg_loss = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        model.zero_grad()  # why we use zero_grad?
        prediction = model(X)
        loss = torch.sqrt(criterion(prediction, Y)).to(device)
        loss.backward()
        optimizer.step()
        avg_loss += loss / len(train_loader)
    print(f'[Epoch: {epoch+1:>2}] Average loss: {avg_loss:.4f}, ', end='')
    
    model.eval()
    criterion.eval()
    with torch.no_grad():
        val_avg_loss = 0.
        for X_val, Y_val in val_loader:
            X_val = X_val.to(device)
            Y_val = Y_val.to(device)
            val_prediction = model(X_val)
            val_loss = torch.sqrt(criterion(val_prediction, Y_val)).to(device)
            val_avg_loss += val_loss / len(val_loader)
        
        print(f"val_loss: {val_avg_loss:.4f}")

[Epoch:  1] Average loss: 4456.9009, val_loss: 4406.0122
[Epoch:  2] Average loss: 4743.9351, val_loss: 4472.6665
[Epoch:  3] Average loss: 4215.2744, val_loss: 4831.9395
[Epoch:  4] Average loss: 4307.5649, val_loss: 4773.2588
[Epoch:  5] Average loss: 3733.3230, val_loss: 4651.1431
[Epoch:  6] Average loss: 3516.4712, val_loss: 4363.0747
[Epoch:  7] Average loss: 3650.0522, val_loss: 4460.0522
[Epoch:  8] Average loss: 3600.7942, val_loss: 5485.5078
[Epoch:  9] Average loss: 3334.7437, val_loss: 3952.6387
[Epoch: 10] Average loss: 3403.0981, val_loss: 4018.1406
[Epoch: 11] Average loss: 3409.8513, val_loss: 24775.2090
[Epoch: 12] Average loss: 3401.1980, val_loss: 3828.3286
[Epoch: 13] Average loss: 3351.7126, val_loss: 4210.9214
[Epoch: 14] Average loss: 3269.0938, val_loss: 3975.8286
[Epoch: 15] Average loss: 3286.4202, val_loss: 4082.4236
[Epoch: 16] Average loss: 3141.9619, val_loss: 3963.2339
[Epoch: 17] Average loss: 3183.3521, val_loss: 3857.0747
[Epoch: 18] Average loss: 3000

[Epoch: 145] Average loss: 636.4312, val_loss: 3550.5588
[Epoch: 146] Average loss: 585.5553, val_loss: 3662.8809
[Epoch: 147] Average loss: 500.2102, val_loss: 3809.3533
[Epoch: 148] Average loss: 499.5533, val_loss: 3600.4363
[Epoch: 149] Average loss: 547.5504, val_loss: 3392.2251
[Epoch: 150] Average loss: 518.5259, val_loss: 3054.8115
[Epoch: 151] Average loss: 525.9615, val_loss: 3753.9622
[Epoch: 152] Average loss: 601.2767, val_loss: 3210.4583
[Epoch: 153] Average loss: 591.2318, val_loss: 3070.2175
[Epoch: 154] Average loss: 593.7047, val_loss: 3417.8110
[Epoch: 155] Average loss: 1189.9629, val_loss: 3135.2729
[Epoch: 156] Average loss: 687.4740, val_loss: 3472.3135
[Epoch: 157] Average loss: 554.5086, val_loss: 3595.0989
[Epoch: 158] Average loss: 558.4708, val_loss: 3920.9575
[Epoch: 159] Average loss: 597.6159, val_loss: 3321.3176
[Epoch: 160] Average loss: 471.7493, val_loss: 3529.9072
[Epoch: 161] Average loss: 403.5014, val_loss: 3304.2891
[Epoch: 162] Average loss: 453

In [9]:
model.eval()
criterion.eval()
ss_tot = 0
ss_res = 0
with torch.no_grad():
    for X_test, Y_test in test_loader:
        X_test = X_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(X_test)
        mean = torch.mean(Y_test)
        ss_tot += torch.sum((Y_test - mean) ** 2)
        ss_res += torch.sum((Y_test - prediction) ** 2)
    
    accuracy = 1 - ss_res/ss_tot
    print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 33.31%
